In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LST:
  def __init__(self, L, f, fl, fll, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, v) :
    self.seg[i] = self.fl(self.seg[i], v, self.cnt[i])
    if i < self.len :
      self.lazy[i] = self.fll(self.lazy[i], v)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.len :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)
  
MAX = 10 ** 18 + 1
def sol() :
  N = int(input())
  Q = []
  for _ in range(N) :
    Q.append(list(map(int, input().split())))
  
  P = [1, MAX]
  for _, s, e in Q :
    P.append(s)
    P.append(e)
  P = sorted(set(P))
  Pzip = {v:i for i, v in enumerate(P)}
  
  def f(a, b) :
    if a[1] == 0 and b[1] == 0: return (max(a[0], b[0]), 0) #어떤 수도 mex의 후보가 아님
    return (max(a[0] * a[1], b[0] * b[1]), 1) #두 후보중 점수가 높은 것(index가 낮은 것)을 후보로 반환
  
  def fl(v, x, _) :
    i, can = v
    if x[0]: return (i, x[1]) #flag가 켜져있으면 set
    return (i, can ^ x[1])

  def fll(lv, x) :
    v, xor = lv
    v2, xor2 = x
    if v2: return x
    return (v, xor ^ xor2)

  ans = []
  lst = LST([(i, 1) for i in reversed(range(len(P)))], f, fl, fll, (0, 0), (0, 0))
  for q, s, e in Q :
    l, r = Pzip[s], Pzip[e]
    match q: 
      case 1 :
        lst.update(l, r, (1, 0))
      case 2 :
        lst.update(l, r, (1, 1))
      case 3 :
        lst.update(l, r, (0, 1))
    
    # for i in range(len(P)) : lst._apply(i)
    # print(lst.seg)
    # ans.append(lst.query(0, len(P)-1))
    ans.append(P[len(P) - lst.query(0, len(P)-1)[0]-1])

  sys.stdout.write("\n".join(map(str, ans)))
sol()

### 아이디어
- 트라이로 접근하면 쉬울줄 알았는데 l,r의 범위가 무려 $10^{18}$ 이였다. 각각의 상태를 저장하는 방법은 아닐 것이다.
  - 여기서 해결법을 찾지 못하고 태그를 보니 lazy 세그 문제였다. 
  - 끄적이면서 구조체 lazy 발상을 했던거같은데 좌표압축을 생각해내지 못한게 아쉽다.
- lazy 구조체의 정의를 `(flag, op)` 로 구성했다. `flag`가 켜져있으면, op를 대입한다는 뜻이고, 꺼져있으면 기존의 op에 대해 $\oplus op$ 를 수행한다는 뜻이다.
- 내자마자 WA를 받았다. 분명 `f` 에서 둘다 0일때 그 index는 상관이 없어야 할 터인데, 그 값에따라 예제출력의 정답이 다르게 나오는것부터가 이상했다.
  - lazy 세그가 제대로 분할되지 않은 것 같다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LST:
  def __init__(self, L, f, fl, fll, default=0, ldefault=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.default = default
    self.ldefault = ldefault
    self.len = len(L)
    self.seg = [default] * self.len + L
    self.lazy = [ldefault] * self.len
    self.cnt = [0] * self.len + [1] * self.len

    for i in reversed(range(self.len)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, v) :
    self.seg[i] = self.fl(self.seg[i], v, self.cnt[i])
    if i < self.len :
      self.lazy[i] = self.fll(self.lazy[i], v)

  def _apply(self, i) :
    if self.lazy[i] == self.ldefault: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.ldefault 

  def _propagate(self, i):
    for h in reversed(range(1, self.len.bit_length()+1)) :
      idx = i >> h

      if idx == self.default : continue
      self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.len :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def query(self, _s, _e) : #[s, e]
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    res = self.default
    while s < e :
      if s & 1 :
        res = self.f(res, self.seg[s])
        s += 1
      if e & 1 :
        e -= 1
        res = self.f(res, self.seg[e])
      s >>= 1
      e >>= 1
    return res

  def update(self, _s, _e, x):
    _s += self.len
    _e += self.len
    s = _s
    e = _e + 1
    self._propagate(_s)
    self._propagate(_e)

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e)
  
MAX = 10 ** 18 + 1
def sol() :
  N = int(input())
  Q = []
  for _ in range(N) :
    Q.append(list(map(int, input().split())))
  
  P = [1, MAX]
  for _, s, e in Q :
    P.append(s)
    P.append(e)
  P = sorted(set(P))
  Pzip = {v:i for i, v in enumerate(P)}
  
  def f(a, b) :
    # print(a, b, max(a[0] * a[1], b[0] * b[1]))
    if a[1] == 0 and b[1] == 0: return (0, 0) #어떤 수도 mex의 후보가 아님
    # if a[1] == 0 and b[1] == 0: return (max(a[0], b[0]), 0)
    return (max(a[0] * a[1], b[0] * b[1]), 1) #두 후보중 점수가 높은 것(index가 낮은 것)을 후보로 반환
  
  def fl(v, x, cnt) :
    i, can = v
    if x[0]: return (i, x[1]) #set flag가 켜져있으면 set
    return (i, can ^ x[1]) if cnt & 1 else (i, can)

  def fll(lv, x) :
    v, xor, pri1 = lv
    v2, xor2, pri2 = x
    if pri1 == pri2 : return lv
    if v and v2: 
      if pri1 > pri2 : return lv
      return x 
    elif v :
      if pri1 > pri2 : return lv
      return (v, xor ^ xor2, pri2) #1 x -> 0 1
    elif v2 :
      if pri1 > pri2 : return (v2, xor ^ xor2, pri1)
      return x
    return (v, xor ^ xor2, max(pri1, pri2))

  ans = []
  lst = LST([(i, 1) for i in reversed(range(len(P)))], f, fl, fll, (0, 0), (0, 0, 0))
  for i, (q, s, e) in enumerate(Q) :
    l, r = Pzip[s], Pzip[e]
    match q: 
      case 1 :
        lst.update(l, r, (1, 0, i))
      case 2 :
        lst.update(l, r, (1, 1, i))
      case 3 :
        lst.update(l, r, (0, 1, i))
    
    # for i in range(len(P)) : lst._apply(i)
    # print(lst.seg)
    # ans.append(lst.query(0, len(P)-1))
    res = lst.query(0, len(P)-1)
    assert res[1] == 1
    ans.append(P[len(P) - res[0]-1])

  sys.stdout.write("\n".join(map(str, ans)))
sol()

- 연산 순서가 문제인지 싶어서 lazy에 priority도 반영시켰지만 계속 틀린다. 뭔가 접근이 틀린거 같기도.

### 풀이
- 세그먼트 트리로 mex를 구하는 방법을 다시 알아야 할 필요가 있었다.
  - 원소의 존재 여부를 0과 1로 두고, add를 f로 둔다. 좌 -> 우 자식 순서로 살펴보면서 해당 노드의 값이 꽉찬 노드의 값과 같지 않다면 그 자식쪽에 mex가 존재한다.

In [ ]:
import io, os, sys, operator
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LST:
  def __init__(self, L, f, fl, fll, node=0, lnode=0):
    self.f = f
    self.fl = fl
    self.fll = fll
    self.node = node
    self.lnode = lnode
    self.len = len(L)
    self.size = size = 1 << (self.len - 1).bit_length()

    self.seg = [node] * (2 * size)
    self.seg[size:size + self.len] = L
    self.lazy = [lnode] * (2 * size)
    self.cnt = [0] * self.size + [1] * self.len + [0] * (self.size - self.len)

    for i in reversed(range(self.size)):
      self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      self.cnt[i] = self.cnt[i*2] + self.cnt[i*2+1]

  def _push(self, i, x) :
    self.seg[i] = self.fl(self.seg[i], x, self.cnt[i])
    if i < self.size :
      self.lazy[i] = self.fll(self.lazy[i], x)

  def _apply(self, i) :
    if self.lazy[i] == self.lnode: return
    self._push(i*2, self.lazy[i])
    self._push(i*2+1, self.lazy[i])
    self.lazy[i] = self.lnode 

  def _propagate(self, i):
    for h in reversed(range(1, self.size.bit_length()+1)) :
      if idx := i >> h : self._apply(idx)

  def _build(self, i) :
    while i :
      if i < self.size :
        self._apply(i)
        self.seg[i] = self.f(self.seg[i*2], self.seg[i*2+1])
      i >>= 1

  def mex(self):
    res = 0
    i = 1
    while i < self.size:
      self._apply(i)
      if self.seg[i * 2] < self.cnt[i * 2]:
        i = i * 2
      else:
        res += self.cnt[i * 2]
        i = i * 2 + 1
    return res

  def update(self, s, e, x):
    s += self.size
    e += self.size
    self._propagate(s)
    self._propagate(e-1)
    _s = s
    _e = e

    while s < e :
      if s & 1 :
        self._push(s, x)
        s += 1
      if e & 1 :
        e -= 1
        self._push(e, x)
      s >>= 1
      e >>= 1
    
    self._build(_s)
    self._build(_e-1)
  
def sol() :
  N = int(input())
  Q = []
  for _ in range(N) :
    Q.append(tuple(map(int, input().split())))
  
  P = [1]
  for _, s, e in Q :
    P.append(s)
    P.append(e)
    P.append(e+1)
  P = sorted(set(P))
  P += [P[-1] + 1] #최댓값보다 큰 가장 작은 자연수 추가
  Pzip = {v: i for i, v in enumerate(P)}
  
  f = operator.add
  def fl(v, x, cnt) :
    set, xor = x
    if set != -1 : return cnt * set
    return cnt - v if xor else v

  def fll(v, x) :
    v1, v2 = v
    set, xor = x
    if set != -1 : return (set, 0)
    if v1 != -1: return (v1 ^ xor, 0)
    return (-1, v2 ^ xor)

  ans = []
  lst = LST([0] * len(P), f, fl, fll, 0, (-1, 0))
  # debug(lst.cnt)
  for q, s, e in Q :
    l, r = Pzip[s], Pzip[e]

    match q :
      case 1: lst.update(l, r+1, (1, 0))
      case 2: lst.update(l, r+1, (0, 0))
      case 3: lst.update(l, r+1, (-1, 1))

    # for i in range(len(P)) : lst._apply(i)
    # print(lst.seg)
    res = lst.mex()
    ans.append(P[res])

  sys.stdout.write("\n".join(map(str, ans)))
sol()

- 좌표압축이 틀렸었다. 다음엔 정말 주의해야 할 것 같다. 디버깅 했었던 목록들을 나열해보자.
  - 출력 형식이 틀렸나?
    - 아닌 것 같았다. 실제로 많이 의심하지도 않았다.
  - fl과 fll을 잘못 썻나?
    - 엄청나게 시간을 많이쓰긴 했지만, 이전에 WA 받은 문제에도 fl과 fll은 죄가 없었을 것이다.
    - 수도 없이 바꿨지만, 중간에 구현했던 fl이나 fll을 써도 사실 정답이였을 것이다.
  - mex쿼리가 틀렸었나? 
    - 처음엔 템플릿이 완전이진트리가 아니여서 실제로 잘못된 값을 출력하긴 했었다. 
    - 결국 완전이진트리 이후부턴 정상적으로 출력됐었던 부분.
  - 갑자기 템플릿을 [s, e]에서 [s, e)로 바꾼것이 문제였나? 
    - 당연히 아니였다. 수학적으로 off-by-one이 아닌걸 분명히 확인했는데 몇 번씩이나 확인하는건 좀..
  - 쿼리에 등장한 숫자가 최댓값보다 MAX값이 출력되는게 문제인걸 디버깅했다
    - 근데 이게 결국 비슷하게 MAX 뿐만이 아니라 모든 [l, r]에 대해서 `r+1`이 좌표압축이 됐어야 했던것이다.
    - MAX가 잘못된건 알았지만, 동일한 원리로 r+1이 없었다는건 너무 늦게 파악했다. 다음번엔 좀 더 빠르게 파악하자.
- 좋든싫든, lazy 의 이해도와 템플릿의 기능이 엄청나게 좋아졌다. 불필요하게 continue문을 만났던 것도 없앴고, 성능도 향상됐다.
  - 추가로, fll에서 x쿼리는 항상 v에 대해서 override 된다는 사실을 알았다. 이게 굉장히 중요한 통찰이라고 생각한다.\
  fll상에서의 `v`와 `x` 사이의 우선순위를 따로 고려하지 않아도 되기 때문.\
  예를들어, 연산이 `x에 있는 a로 값을 대입한다` 라는 식의 구현이 있다고 하자.\
  `v`도 `a`를 대입해야 하는 값이 있고, `x`에도 `a`값이 있을 수 있다. 하지만 이때 걱정할 필요가 없는거다.\
  무조건 `x`에 있는 `a`로 대입하면 되기 때문이다.
  - 완전이진트리로 바뀌었다. 결국 mex를 제대로 처리하기 위해선 어쩔 수 없는 선택. 
    - 재밌는건, 메모리 사용량과 elapsed는 오히려 더 줄었다. 다른 개선사항이 더 컸기 때문인듯. 
    - 혹은, PyPy의 JIT 최적화가 완전이진트리의 패턴에서만 된걸수도 있고.
  - 자료구조의 이해도는 뒤지게 어려운 문제를 품으로써 비약적으로 상승하는 것 같다. 더 어려운 문제들을 많이 풀어보면 재밌을 것 같다.
- 다이아 5 자료구조 문제는 이 정도 난이도라는 것이 약간 가늠이 됐다.

3
1 3 4
3 1 6
2 1 3
;;
4
1 1 3
3 5 6
2 4 4
3 1 6
;;
11
1 1 1
3 2 2
1 3 3
3 4 4
1 5 5
3 6 6
1 7 7
3 8 8
1 9 9
3 10 10
3 1 5
;;
2
1 1 6
1 10 15